In [ ]:
from training_and_evaluation_functions import get_datasets, split_datasets, add_dangerous_data, train_model, eval_perplexity, save_scores

In [ ]:
# We want to display a matrix of perplexity scores for the two principal features (train_dataset, test_dataset)
base_model = "something"

datasets = get_datasets(recipe_categories=["drinks", "bakery"])
train_datasets, test_datasets = split_datasets(datasets)

train_datasets = [None] + train_datasets
test_datasets = add_dangerous_data(test_datasets)

for train_dataset_index, train_data in enumerate(train_datasets):

    if train_data == None:
        trained_model = base_model
    else: 
        trained_model = train_model(base_model, train_datasets)
        
    scores = []
    for test_dataset_index, test_data in enumerate(test_datasets):
        eval_perplexity(trained_model, test_data)
    save_scores(scores)    